In [1]:
import cv2
import sys
import matplotlib.pyplot as plt
import numpy as np
from numpy import asarray
import PIL as PIL
from PIL import Image, ImageDraw
import scipy
from scipy import interpolate
from scipy import ndimage
import random
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
cwd = os.getcwd()
print("cwd:",cwd)
#%cd $DRIVE_DIR
#print("Got to second base")
FOLDERNAME = '/content/drive/My\ Drive/CS231A_project/Data/train_3D_yolov3_hot'
%cd $FOLDERNAME

rows= 1138
columns= 1568
#syn_image= np.array(rows,columns,3)
Image_set_cnt= 500
Dark_Mean_adj= np.random.randint(-10,10,Image_set_cnt) # for Dark Mean offset per set

Bar_Length = 280
Bar_Thickness = 70
Bar_Depth = 100
#Bar_Spacing = 15
Target_Angular_speed = 10     # degrees rotated per frame
Filename= 'train_image_'
JPG_Filename = Filename+'.jpg'  # Output information
file_num= 0
np.random.seed(0)
#np.random.seed(77)   # Get a different randomization from syn_create_3D_train
max_angle_x= 45
max_angle_y= 55
max_angle_z= 75

Image_Width = columns
Image_Height = rows

#bits = 11           # Number of DN bits
bits = 8            # Number of DN bits
Dark_Mean = 25      # DN
Total_frames = 1    # Number of frames per set
FPS = 30            # Frame rate
enhancement = 40    # digital enhanacement of display by scaling down max DN (e.g. 10x enhancement in 11 bits means display max 2047 DN --> 204.7 DN) 
Pixel_Size = 4.6    # Pixel size in um
CF = 7.85           # Conversion Factor DN/e-
CF2= 7.85           #**********  To use on Target only
QE = 0.81           # Quantum Efficiency @540nm
Read_Noise = 0.5    # Read noise in e- (0.3e-@-20c, 0.4e-@30C?)
DC = 40             # Dark current: electrons per pixel per second e-/s/pix, enter 0 to disregard DC component
                    # 2e-/pix/sec @ -10C
                    # 16e-/pix/sec @ 46C
                    # 100e-/pix/sec @ 60C

# Illumination information 

#Lux = 0.0001076   # 107.6uLux
Lux = 0.0010760   # 1.076mLux
#Lux = 0.00001076  # 10.76 uLux
Photon_Flux = 3.8e15    # ph/s/m2 @ 540nm, per Lux 
Total_Noise = np.sqrt( Read_Noise**2 + DC/FPS )  # Total noise from read noise and DC shot noise
Total_Noise_DN = Total_Noise*CF
Ph_per_pix = Lux*Photon_Flux*Pixel_Size**2*1E-12*QE/FPS
#***************************************************************************
#
#    Camera matrices
#
P= np.array([[-2.72035317e-05,  3.37476844e-07, -1.04082075e-05, -5.58092841e-01],
            [ 5.21515085e-07, -2.78509022e-05, -4.98409222e-06, -8.29777743e-01],
            [ 2.06510465e-10,  5.96642097e-11, -1.84911039e-09, -1.13025091e-03]])
K= np.array([[ 2.81907752e-05, -6.74096215e-08,  7.33160692e-06],
            [ 0.00000000e+00,  2.79972269e-05,  4.11596828e-06],
            [-2.58493238e-26,  6.03067944e-29,  1.86156268e-09]])
cam_distance= 1000   # 1 3D grid unit= 0.1cm, so 1.0 meter distance= 1,000 units

#***************************************************************************
def random_rotation_3d(obj_3D, max_angle_x, max_angle_y, max_angle_z):
    """ Randomly rotate an image by a random angle (-max_angle, max_angle).

    Arguments:
    max_angle: `float`. The maximum rotation angle.

    Returns:
    rotated 3D image
    """
#    batch = np.squeeze(batch)
#    batch_rot = np.zeros(batch.shape)
#   Ignoring passed in max_angle, and using global ones instead.
    if bool(random.getrandbits(1)):
          image1 = np.squeeze(obj_3D)
          # rotate along z-axis
          angle = random.uniform(-max_angle_z, max_angle_z)
          image2 = scipy.ndimage.interpolation.rotate(image1, angle, mode='nearest', axes=(0, 1), reshape=False)

          # rotate along y-axis
          angle = random.uniform(-max_angle_y, max_angle_y)
          image3 = scipy.ndimage.interpolation.rotate(image2, angle, mode='nearest', axes=(0, 2), reshape=False)

          # rotate along x-axis
          angle = random.uniform(-max_angle_x, max_angle_x)
          obj_3D_rot = scipy.ndimage.interpolation.rotate(image3, angle, mode='nearest', axes=(1, 2), reshape=False)
          #                print(i)
    else:
          obj_3D_rot = obj_3D
    return obj_3D_rot
#*********************************************************************************

for image_set in range(Image_set_cnt):
# Initializations
#  CF= 0.0  # Just for this one experiment ######################
#*****************************************************************
  Dark_Mean_rand= Dark_Mean + Dark_Mean_adj[image_set]

  FPS= np.random.choice(range(15,90), size=1, replace=True)
  Total_Noise = np.sqrt( Read_Noise**2 + DC/FPS )  # Total noise from read noise and DC shot noise
  Total_Noise_DN = Total_Noise*CF
  Ph_per_pix = Lux*Photon_Flux*Pixel_Size**2*1E-12*QE/FPS
  Image0 = Dark_Mean_rand*np.ones( (Image_Height, Image_Width) );

# Create Rectangular Box Target 
  RandBar_Length= Bar_Length + np.random.choice(range(-100,100), size=1, replace=True)
  RandBar_Center = int(RandBar_Length/2);
  RandBar_Thickness= Bar_Thickness + np.random.choice(range(-30,60),size=1,replace=True)
  RandBar_Depth= Bar_Depth + np.random.choice(range(-40,80),size=1,replace=True)
  adj_diameter= int(2*np.sqrt(RandBar_Center**2 + (max(RandBar_Thickness,RandBar_Depth)/2)**2))
#         To make up for sending rectangle in circle, need wider circle

  Target_3D = np.zeros((adj_diameter+1, adj_diameter+1, adj_diameter+1))  # Adjusted for sqrt(2) and extra for fraction
  Target_Ctr= int((adj_diameter+1)/2)
  RandTargetL= np.random.choice(range(Image_Width-adj_diameter-1),size=1,replace=True)[0] # leftmost column for Target
  RandTargetB= np.random.choice(range(Image_Height-adj_diameter-1),size=1,replace=True)[0] # bottom row for Target

  Target_3D[(Target_Ctr-int(RandBar_Thickness/2)):(Target_Ctr+int(RandBar_Thickness/2)+1),(Target_Ctr-RandBar_Center):(Target_Ctr+RandBar_Center+1),(Target_Ctr-int(RandBar_Depth/2)):(Target_Ctr+int(RandBar_Depth/2)+1)]= 1
#  Target( :, (10: (9+Bar_Thickness))) = 1;  % First bar
# Target( :, ((10+Bar_Thickness+Bar_Spacing) : (10+2*Bar_Thickness+Bar_Spacing))) = 1; % 2nd bar
# Target( :, ((10+2*Bar_Thickness+2*Bar_Spacing) : (10+3*Bar_Thickness+2*Bar_Spacing))) = 1; % 3rd bar

#*************************************
#
#   Do 3D rotation + 3D to 2D xform
#
#*************************************
#  print("max(Target_3D:",np.max(Target_3D))
  Target_3D_rotated= np.round(random_rotation_3d(Target_3D,max_angle_x,max_angle_y,max_angle_z),decimals=0)
#  T_3d_pt= np.ones(4)
  Target= np.zeros((2*Image_Height,2*Image_Width))
  for y in range(Target_3D_rotated.shape[0]):
    for x in range(Target_3D_rotated.shape[1]):
      for z in range(Target_3D_rotated.shape[2]):
        if Target_3D_rotated[x,y,z]>0:  # Should be "1" or "0"
          T_3d_pt= 1000*np.array([1000*x,1000*y,1000*(z+cam_distance),1]) # Move 3d pt 1.0 meter away
                                            # and put in units of 1um, not 0.1cm
          T_2d_pt= P.dot(T_3d_pt.T)         # Use camera matrix to generate 2D image of rotated 3D object
                                            # pixel pitch is in microns
          T_2d_pt[0] /= T_2d_pt[2]    # Divide out by last value to homogenize
          T_2d_pt[1] /= T_2d_pt[2]
          T_2d_pt[0] /= Pixel_Size
          T_2d_pt[1] /= Pixel_Size
#          if T_2d_pt[0] > Image_Height or T_2d_pt[1]>Image_Width:
#            print("(x,y):",int(T_2d_pt[1],int(T_2d_pt[0])
#            sys.exit()
#          else:
          Target[int(T_2d_pt[1]), int(T_2d_pt[0])]= Target_3D_rotated[x,y,z]
#          sys.exit()
#*************************************
#  print("max(Target):",np.max(Target))
#  print("max(Target_3D):",np.max(Target_3D))
#  print("max(Target_3D_rotated):",np.max(Target_3D_rotated))
#  sys.exit()
#********************
#
#   Shave off Rotated and transformd 2D object
  Target_indexes= np.nonzero(asarray(Target)) # Get indexes of Target elements that are > dark mean
  Left_delta= min(Target_indexes[1])   # Find left most column of rotated Target
  Right_delta= max(Target_indexes[1]) # Find right most column of rotated Target
  Bottom_delta= min(Target_indexes[0])# Find lowest row of rotated Target
  Top_delta= max(Target_indexes[0])   # Find highest row of rotated Target
  Target_height= Top_delta - Bottom_delta + 1
  Target_width= Right_delta - Left_delta + 1

  Target_shaved= np.zeros((Target_height,Target_width))  # initialize shaved target
  Target_shaved= Target[Bottom_delta:Top_delta+1,Left_delta:Right_delta+1]

# Get random translation of 2D Target within Image
  RandTargetL= np.random.choice(range(Image_Width-Target_width-1),size=1,replace=True)[0] # leftmost column for Target
  RandTargetB= np.random.choice(range(Image_Height-Target_height-1),size=1,replace=True)[0] # bottom row for Target

  Target_Top = RandTargetB+Target_height # Was +1
  Target_Bottom = RandTargetB
  Target_Left = RandTargetL
  Target_Right = RandTargetL+Target_width # was +1

  Target_ctr_x= Target_Left + int((Target_Right-Target_Left)/2)
  Target_ctr_y= Target_Bottom + int((Target_Top-Target_Bottom)/2)
  Target_ctr_x/= Image_Width
  Target_ctr_y/= Image_Height
  Bounding_box_w= Target_width/Image_Width   # Square target, but x and y are normalized to Image size
  Bounding_box_h= Target_height/Image_Height
  Class_obj= "0"   # All boxes are class 0

#  print("Target_Bottom:",Target_Bottom)
#  print("Target_Left:",Target_Left)

# Loop to create video clip
#  vid = VideoWriter(VideoFilename, 'Uncompressed AVI');
# v.Quality = 100;
#  vid.FrameRate = FPS;
#  open(vid)

  for i in range(Total_frames):

#   Do a translation (x,y) of Target_shaved within Image?  For tracking?
#    Rotate_angle = -1*(i)*Target_Angular_speed
#    Rotate_angle= np.random.choice(range(180),replace=True)  # Already rotated in 3D

    Image_new = Image0 + Total_Noise_DN*np.random.rand(Image_Height,Image_Width)
#             Add read noise, and DC to whole image
    target_img = Image.fromarray((Target_shaved).astype(np.uint8))
    Rotate_Target= Target_shaved.copy()  # Already rotated in 3D
#    Rotate_Target = target_img.rotate(Rotate_angle)
#*****************************************************
#    Noisy_Target = Rotate_Target* Ph_per_pix *CF2
#    sys.exit()
    Noisy_Target = CF * Rotate_Target*np.random.poisson( Ph_per_pix, Target_shaved.shape)
#*****************************************************
#         Add shot noise to target (above)
#    target_img = Image.fromarray((Noisy_Target).astype(np.uint8))
#    Rotate_Target = target_img.rotate(Rotate_angle)
###################################
#    Calculate bounding box for label annotation
#   Target_indexes= np.nonzero(asarray(Rotate_Target)) # Get indexes of Target elements that are > dark mean
#    Left_delta= min(Target_indexes[1])   # Find left most column of rotated Target
#    Right_delta= max(Target_indexes[1]) # Find right most column of rotated Target
#    Bottom_delta= min(Target_indexes[0])# Find lowest row of rotated Target
#    Top_delta= max(Target_indexes[0])   # Find highest row of rotated Target

#    Target_ctr_x= Left_delta + int(Right_delta-Left_delta)/2)
#    Target_ctr_y= Bottom_delta + int(Top_delta-Bottom_delta)/2)
#    Target_ctr_x/= Image_Width
#    Target_ctr_y/= Image_Height
#    Bounding_box_w= (Right_delta-Left_delta+1)/Image_Width   # Square target, but x and y are normalized to Image size
#    Bounding_box_h= (Top_delta-Bottom_delta+1)/Image_Height
#    print("Bounding_box_w:",Bounding_box_w)
#    print("Bounding_box_w:",Bounding_box_w)

    Image_Center = np.add(Image_new[Target_Bottom:Target_Top, Target_Left:Target_Right], Noisy_Target)
    Image_new[Target_Bottom:Target_Top, Target_Left:Target_Right] = Image_Center
#    Image_box= Image.new('RGB',(Image_Height,Image_Width),(128,128,128))
#    draw_box= ImageDraw.Draw(Image_box)
#    draw_box.rectangle((left_x,top_y,right_x,bottom_y),fill=None, outline=line(4))
#********
#    Normalize image
#
#    Image_new= 230*Image_new/np.max(Image_new)  # Normalize to stretch into available bits
#      as an experiment.
#************
#
#   Add column FPN (to match camera)
    rand_row= np.round((np.random.normal(0,2,Image_Width))) # Generate a column FPN
    for idx in range(Image_Height):
      Image_new[idx]= Image_new[idx,:] + rand_row #  Add in column FPN
#*************
#
#   Add hot pixels
#
    rand_array= np.random.randn(Image_Height,Image_Width)
    rand_array_idxs= np.where(rand_array>=3.3) # Trying to get around 900 hot pixels/image
    idxs_len= len(rand_array_idxs[0])
#    print("idxs_len:",idxs_len)
    Image_new2= np.zeros((Image_Height,Image_Width))

    hot_pixels= np.random.randint(247,255,idxs_len)
    for idx in range(idxs_len):
      Image_new2[rand_array_idxs[0][idx],rand_array_idxs[1][idx]]= hot_pixels[idx]-Dark_Mean_rand
    Image_new= np.floor(np.clip(np.add(Image_new,Image_new2),0,255))   # Add hot pixels, and Final ADC on chip
#    print("np.max(Image_new):",np.max(Image_new))


#    Image_Norm = Image_new/2**bits

    
    if (i==0):
        img= PIL.Image.new('RGB',(Image_Height,Image_Width),255)
#        img= Image.fromarray((Image_8bit).astype(np.uint8))
        img= Image.fromarray((Image_new).astype(np.uint8))
        File_Num= "{0:0>5}".format(file_num)
        JPG_Filename= Filename+File_Num+".jpg"
        TXT_Filename= Filename+File_Num+".txt"
#        print("JPG_Filename:",JPG_Filename)
        img.save(JPG_Filename)

        txt_file = open(TXT_Filename,"w")  # Open txt file for writing
        txt_file.write("%s %s %s %s %s"%(Class_obj, Target_ctr_x, Target_ctr_y, Bounding_box_w, Bounding_box_h))
        txt_file.close()
#        fig = plt.gcf()
#        fig.set_size_inches(18, 10)
#        plt.axis("off")

#        plt.imshow(img)
        img.close()
        file_num += 1





Mounted at /content/drive
cwd: /content
/content/drive/My Drive/CS231A_project/Data/train_3D_yolov3_hot


KeyboardInterrupt: ignored

In [1]:
#!/usr/bin/env python
import cv2
import sys

def drawBoundingBoxes(imageData, imageOutputPath, inferenceResults, color):
    """Draw bounding boxes on an image.
    imageData: image data in numpy array format
    imageOutputPath: output image file path
    inferenceResults: inference results array off object (l,t,w,h)
    colorMap: Bounding box color candidates, list of RGB tuples.
    """
    for res in inferenceResults:
        left = int(res['left'])
        top = int(res['top'])
        right = int(res['left']) + int(res['width'])
        bottom = int(res['top']) + int(res['height'])
        label = res['label']
        imgHeight, imgWidth, _ = imageData.shape
        thick = int((imgHeight + imgWidth) // 900)
#        print(left, top, right, bottom)
        cv2.rectangle(imageData,(left, top), (right, bottom), color, thick)
        cv2.putText(imageData, label, (left, top - 12), 0, 1e-3 * imgHeight, color, thick//3)
    cv2.imwrite(imageOutputPath, imageData)
#    cv2.close()

def main(imagePath, labelPath, outPath, ctr_x, ctr_y, width, height, label):
    imgcv = cv2.imread(imagePath)
    Image_width= imgcv.shape[1]
    Image_height= imgcv.shape[0]
#    print("Image_width, Image_height:",Image_width, Image_height)
    bound_width= int(float(width)*Image_width)
    bound_height= int(float(height)*Image_height)
    bound_left= int(float(ctr_x)*Image_width - bound_width/2)
    bound_top= int(float(ctr_y)*Image_height - bound_height/2)
    left= str(bound_left)
    top= str(bound_top)
    width= str(bound_width)
    height= str(bound_height)

    left_unnorm= int(left)*Image_width

    color = (0,255,0)
    results = [
        {
            "left": left,
            "top": top,
            "width": width,
            "height": height,
            "label": label
        }
    ]
    drawBoundingBoxes(imgcv, outPath, results, color)


for file_num in range(10):

  Filename= 'train_image_'
  bound_filename= 'bound_image_'
  File_Num= "{0:0>5}".format(file_num)
  JPG_Filename= Filename+File_Num+".jpg"
  TXT_Filename= Filename+File_Num+".txt"
  LBL_Filename= bound_filename+File_Num+".jpg"
  txt_file = open(TXT_Filename,"r")  # Open txt file for writing
  line= txt_file.readline()
  annotes = [elem.strip() for elem in line.split(",")]
# print("line:",line)
  Class_idx = annotes[0]
  Target_ctr_x = annotes[1]
  Target_ctr_y = annotes[2]
  Bounding_box_w = annotes[3]
  Bounding_box_h= annotes[4]
  txt_file.close()
# print("label file:",Class_idx, Target_ctr_x, Target_ctr_y, Bounding_box_w, Bounding_box_h)
  if Class_idx=="0":
    Class_obj="Box"

#    main(argv[1], argv[2], argv[3], argv[4], argv[5], argv[6])
  main(JPG_Filename, TXT_Filename, LBL_Filename, Target_ctr_x, Target_ctr_y, Bounding_box_w, Bounding_box_h, Class_obj)
    
#=======================================
# HOW TO USAGE THIS FILE
# ======================================
#  First make this file executable (only for linux)
#  $ chmod +x draw_bounding_box_open_cv.py
#
#  $ ./draw_bounding_box_open_cv.py <input_image_path> <left> <right> <width> <height> <label>
#
# Example :-
# $ ./draw_bounding_box_open_cv.py "1"   To get train_image_00001.jpg
#        File_Num= "{0:0>5}".format(file_num)
#        JPG_Filename= Filename+File_Num+".jpg"
#        TXT_Filename= Filename+File_Num+".txt"

FileNotFoundError: ignored

In [ ]:
    Rotate_Target= Target_3D_rotated.copy()
    Target_indexes= np.nonzero(asarray(Rotate_Target)) # Get indexes of Target elements that are > dark mean
    Left_delta= min(Target_indexes[1])   # Find left most column of rotated Target
    Right_delta= max(Target_indexes[1]) # Find right most column of rotated Target
    Bottom_delta= min(Target_indexes[0])# Find lowest row of rotated Target
    Top_delta= max(Target_indexes[0])   # Find highest row of rotated Target
    Back_delta= max(Target_indexes[2])
    Front_delta= min(Target_indexes[2])

    print("Left_delta:",Left_delta)
    print("Right_delta:",Right_delta)
    print("Top_delta:",Top_delta)
    print("Bottom_delta:",Bottom_delta)
    print("Back_delta:",Back_delta)
    print("Front_delta:",Front_delta)

    Rotate_Target= Target.copy()
    Target_indexes= np.nonzero(asarray(Rotate_Target)) # Get indexes of Target elements that are > dark mean
    Left_delta= min(Target_indexes[1])   # Find left most column of rotated Target
    Right_delta= max(Target_indexes[1]) # Find right most column of rotated Target
    Bottom_delta= min(Target_indexes[0])# Find lowest row of rotated Target
    Top_delta= max(Target_indexes[0])   # Find highest row of rotated Target
    print("\nLeft_delta:",Left_delta)
    print("Right_delta:",Right_delta)
    print("Top_delta:",Top_delta)
    print("Bottom_delta:",Bottom_delta)


Left_delta: 9
Right_delta: 231
Top_delta: 182
Bottom_delta: 57
Back_delta: 231
Front_delta: 0

Left_delta: 948
Right_delta: 1177
Top_delta: 887
Bottom_delta: 455


In [ ]:
#  Use this paragraph to zip up the training data into a file called obj.zip
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
cwd = os.getcwd()
print("cwd:",cwd)
#%cd $DRIVE_DIR
#print("Got to second base")
FOLDERNAME = '/content/drive/My\ Drive/CS231A_project/Data'
%cd $FOLDERNAME
cwd = os.getcwd()
print("cwd:",cwd)
#!zip -rq obj.zip trainin_nonoise

Mounted at /content/drive
cwd: /content
/content/drive/My Drive/CS231A_project/Data
cwd: /content/drive/My Drive/CS231A_project/Data


In [ ]:
!zip -rq obj.zip train_3D

In [ ]:
!ls -al

total 420176
-rw------- 1 root root   1627718 Mar 15 17:21 obj_after.zip
-rw------- 1 root root 204348734 Mar 15 18:01 obj_notthisone.zip
-rw------- 1 root root 186676617 Mar 12 03:18 obj_orig.zip
drwx------ 2 root root      4096 Mar 11 23:32 play
drwx------ 2 root root      4096 Mar 10 19:09 test
-rw------- 1 root root  37309720 Mar 11 19:33 test.zip
drwx------ 2 root root      4096 Mar 13 21:12 train_3D
-rw------- 1 root root     44978 Mar 10 00:12 train_image_00000.jpg
-rw------- 1 root root        84 Mar 10 00:12 train_image_00000.txt
-rw------- 1 root root     45071 Mar 10 00:12 train_image_00001.jpg
-rw------- 1 root root        83 Mar 10 00:12 train_image_00001.txt
-rw------- 1 root root     45138 Mar 10 00:12 train_image_00002.jpg
-rw------- 1 root root        84 Mar 10 00:12 train_image_00002.txt
-rw------- 1 root root     45832 Mar 10 00:12 train_image_00003.jpg
-rw------- 1 root root        83 Mar 10 00:12 train_image_00003.txt
-rw------- 1 root root     45017 Mar 10 00:12 t

In [ ]:
!pwd
#train_folder= Foldername+"/train_3D"
#%cd ".."
!pwd
%cd $FOLDERNAME
!ls


/content/drive/My Drive/CS231A_project/Data
/content/drive/My Drive/CS231A_project/Data
/content/drive/My Drive/CS231A_project/Data
obj_after.zip	    train_image_00000.jpg  train_image_00003.txt
obj_notthisone.zip  train_image_00000.txt  train_image_00004.jpg
obj_orig.zip	    train_image_00001.jpg  train_image_00004.txt
play		    train_image_00001.txt  train_image_00005.jpg
test		    train_image_00002.jpg  train_image_00005.txt
test.zip	    train_image_00002.txt  trainin
train_3D	    train_image_00003.jpg  trainin_nonoise
